In [ ]:
import numpy as np
import pandas as pd
import plotnine
from plotnine import ggplot, geom_histogram, aes, stat_smooth, facet_wrap
import sbn

plotnine.options.figure_size = (9, 6)

In [ ]:
inst = sbn.instance("mb")
inst.read_nexus_file("../_ignore/mb/primates/primates_out.t")
inst.process_loaded_trees()

In [ ]:
burn_in = 10
mb_branch_lengths = pd.DataFrame(
    np.array([np.array(a) for a in inst.branch_lengths_by_split()])[:, burn_in:].transpose())

In [ ]:
mb_branch_lengths

In [ ]:
(ggplot(pd.melt(mb_branch_lengths, var_name="branch"), aes('value'))
 + geom_histogram()
 + facet_wrap('branch'))